In [1]:
import requests
from bs4 import BeautifulSoup

## Những tag chứa link báo theo ngày

In [3]:
#### Class of links in The Guardian
#fc-slice__item l-row__item l-row__item--span-1 u-faux-block-link
#fc-slice__item l-list__item l-row__item l-row__item--span-1 u-faux-block-link
#fc-show-more--hide-on-mobile js-hide-on-mobile fc-slice__item l-list__item l-row__item l-row__item--span-1 u-faux-block-link

## Thu thập link thử của 1 ngày

In [10]:
test = requests.get("https://www.theguardian.com/artanddesign/2002/feb/08/all")
print(test)

<Response [200]>


In [11]:
soup = BeautifulSoup(test.text, 'html.parser')
print(soup)


<!DOCTYPE html>

<html class="js-off is-not-modern id--signed-out" data-page-path="/artanddesign/2002/feb/08/all" id="js-context" lang="en">
<head>
<title>Art and design | Artanddesign | The Guardian</title>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible">
<meta content="telephone=no" name="format-detection"/>
<meta content="True" name="HandheldFriendly"/>
<meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
<link href="https://assets.guim.co.uk/" rel="dns-prefetch"/>
<link href="https://i.guim.co.uk" rel="dns-prefetch"/>
<link href="https://api.nextgen.guardianapps.co.uk" rel="dns-prefetch"/>
<link href="https://hits-secure.theguardian.com" rel="dns-prefetch"/>
<link href="//j.ophan.co.uk" rel="dns-prefetch"/>
<link href="//ophan.theguardian.com" rel="dns-prefetch"/>
<link href="//phar.gu-web.net" rel="dns-prefetch"/>
<link href="https://assets.guim.co.uk/" rel="preconnect"/>
<link href="https://i.guim.co.uk" rel="preconnec

## Function để crawl báo của 1 ngày từ 1 chủ đề về

In [17]:
import requests
from bs4 import BeautifulSoup
test = requests.get("https://www.theguardian.com/artanddesign/2003/feb/09/all")
soup = BeautifulSoup(test.text, 'html.parser')
# Function to process links
def process_links(links):
    for link in links:
        a = link.find("a")
        href = a["href"]
        # Check if '/video/' or '/live/' is in the link
        if '/video/' not in href and '/live/' not in href:
            print(href)

# First set of links
links = soup.find_all("li", class_="fc-slice__item l-row__item l-row__item--span-1 u-faux-block-link")
process_links(links)
print("--------------------")

# Second set of links
otherLinks = soup.find_all("li", class_="fc-slice__item l-list__item l-row__item l-row__item--span-1 u-faux-block-link")
process_links(otherLinks)
print("--------------------")

# Third set of links
otherLinks = soup.find_all("li", class_="fc-show-more--hide-on-mobile js-hide-on-mobile fc-slice__item l-list__item l-row__item l-row__item--span-1 u-faux-block-link")
process_links(otherLinks)

https://www.theguardian.com/artanddesign/2003/feb/08/art.artsfeatures
https://www.theguardian.com/artanddesign/2003/feb/08/architecture.artsfeatures
--------------------
--------------------


# Không đổi code này nữa

In [79]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def process_links(links, hrefs):
    for link in links:
        a = link.find("a")
        href = a["href"]
        if '/video/' not in href and '/live/' not in href:
            hrefs.append(href)

hrefs = []
topic = ['world', 'uk-news', 'environment', 'science', 'global-development', 'football', 'technology', 'business', 'obituaries', 'sport', 'books', 'music', 'tv-and-radio', 'artanddesign', 'film', 'games', 'lifeandstyle','fashion', 'travel', 'money']
day = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
month = ["jan", 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct','nov','dec']
year = [i for i in range(2016, 2023)][::-1]

year =[2023]
month = ['nov']
day = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']

retry_attempts = 3
retry_delay = 5  # seconds
for t in topic:
    for y in year:
        for m in month:
            for d in day:
                attempts = 0
                while attempts < retry_attempts:
                    try:
                        url = f"https://www.theguardian.com/{t}/{y}/{m}/{d}/all"
                        response = requests.get(url, timeout=30).text
                        soup = BeautifulSoup(response, 'html.parser')
                        # Process links
                        links = soup.find_all("li", class_="fc-slice__item l-row__item l-row__item--span-1 u-faux-block-link")
                        process_links(links, hrefs)
                        # ... Process other link types
                        break  # Break the loop if successful
                    except requests.exceptions.ConnectionError:
                        print(f"Connection error for {url}. Retrying...")
                        attempts += 1
                        time.sleep(retry_delay)
                    except requests.exceptions.RequestException as e:
                        print(f"Request failed: {e}")
                        break

# Export to CSV
with open('guardian_links_nov.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for href in hrefs:
        writer.writerow([href])

print(f"Total links (excluding videos and live news): {len(hrefs)}")


Connection error for https://www.theguardian.com/sport/2023/nov/16/all. Retrying...
Connection error for https://www.theguardian.com/travel/2023/nov/06/all. Retrying...
Total links (excluding videos and live news): 1554


# Clean the data

## Remove the links /gallery/, because it is not actually a news
## Remove duplicates

In [82]:
import csv

file_path = 'guardian_links_2015_2023.csv'

def read_and_clean_csv(file_path):
    cleaned_links = []
    seen = set()  # Use a set to keep track of seen links for efficiency

    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            link = row[0]
            if '/gallery/' not in link and link not in seen:  # Filter out /gallery/ links and duplicates
                cleaned_links.append(link)
                seen.add(link)
    
    return cleaned_links

def write_cleaned_csv(file_path, cleaned_links):
    with open(file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for link in cleaned_links:
            writer.writerow([link])  # Write each cleaned link back to the file

# Read, clean, and write back to the CSV
cleaned_links = read_and_clean_csv(file_path)
write_cleaned_csv(file_path, cleaned_links)

print(f"Updated CSV file with {len(cleaned_links)} unique links (excluding /gallery/ links).")


Updated CSV file with 188002 unique links (excluding /gallery/ links).


# Crawl from a link

In [128]:
response = requests.get("https://www.theguardian.com/world/2023/sep/01/german-iraqi-woman-charged-with-killing-lookalike-to-fake-her-own-death").text
soup = BeautifulSoup(response, 'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="en">
<head>
<!-- Hello there, HTML enthusiast! -->
<title>German-Iraqi woman charged with killing lookalike to fake her own death | Germany | The Guardian</title>
<meta content="Woman accused of contacting beauty blogger on Instagram and offering free salon treatment before killing her with accomplice" name="description"/>
<link href="https://www.theguardian.com/world/2023/sep/01/german-iraqi-woman-charged-with-killing-lookalike-to-fake-her-own-death" rel="canonical"/>
<meta charset="utf-8"/>
<meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
<meta content="#052962" name="theme-color">
<link href="https://assets.guim.co.uk/static/frontend/manifest.json" rel="manifest"/>
<link href="https://assets.guim.co.uk/static/frontend/icons/homescreen/apple-touch-icon.svg" rel="apple-touch-icon" sizes="any"/>
<link href="https://assets.guim.co.uk/static/frontend/icons/homescreen/apple-touch-icon-512.png" rel="apple-touch-icon" sizes="5

In [129]:
# Extract title - primary method
title_container = soup.find(attrs={"data-gu-name": "headline"})
title_text = title_container.find('h1').get_text() if title_container and title_container.find('h1') else ''

# Extract title - alternative method if primary method fails
if not title_text:
    og_title = soup.find('meta', attrs={"property": "og:title"})
    title_text = og_title['content'] if og_title and 'content' in og_title.attrs else ''
print(title_text)

German-Iraqi woman charged with killing lookalike to fake her own death


In [111]:
meta_description = soup.find('meta', attrs={"name": "description"})

if meta_description and 'content' in meta_description.attrs:
    description_content = meta_description['content']

    # Check if the description contains HTML tags
    if '<' in description_content and '>' in description_content:
        # Decode HTML entities and parse the result with BeautifulSoup
        decoded_description = html.unescape(description_content)
        description_soup = BeautifulSoup(decoded_description, 'html.parser')
        description_text = description_soup.get_text()
    else:
        description_text = description_content

    print(description_text)
else:
    print(0)

Manager says his team ‘feels like we’re a bit closer’ to champions as he looks to end Guardiola’s 23-game winning streak at home


## Lấy content bỏ phần đuôi ở cuối, bỏ phần 'newsletter promotion figure' trong một số bài đăng

In [112]:
content = soup.find(attrs={"data-gu-name": "body"})

if content:
    # Find and remove the newsletter promotion figure
    newsletter_promotion = content.find('figure', class_='dcr-173mewl')
    if newsletter_promotion:
        newsletter_promotion.decompose()

    # Get text after removing the unwanted part
    main_content = content.get_text()

    # Optionally split at "Explore more on these topics" if you still want this part removed
    main_content = main_content.split("Explore more on these topics")[0]

    print(main_content.strip())
else:
    print("Content not found")

Jürgen Klopp has described Liverpool’s top-of-the-table encounter with Manchester City as a game the whole planet should be watching, but believes the result will not define the trajectory of his team.First host second in the Premier League on Saturday lunchtime as Liverpool travel to the Etihad Stadium hoping to end City’s 23-match winning run at home in all competitions.Klopp has the superior head-to-head record against Pep Guardiola, with 12 wins from 28 meetings to Guardiola’s 11, and takes an improved Liverpool side down the M62 with renewed confidence.Pep Guardiola: I won’t resign if Manchester City are relegated to League OneRead moreA Liverpool win would send them two points clear of City and relight the fuse on a Premier League rivalry that produced fireworks between 2019 and 2022 but simmered down last season as Klopp’s team, who underwent something of a transition, finished fifth. Taking on City once again is an exciting prospect for the German, who says his new-look team ar

In [113]:
# First method (assuming the structure with "data-gu-name" : "meta")
meta = soup.find(attrs={"data-gu-name": "meta"})
date = None
if meta:
    date = meta.find("span", class_="dcr-u0h1qy")
    if date:
        date = date.text

# Second method (assuming the structure with class "dcr-18bzkx3")
if not date:
    date = soup.find("div", class_="dcr-18bzkx3")
    if date:
        date = date.text

# Output the date or a message if not found
if date:
    print(date)
else:
    print(0)


Fri 24 Nov 2023 22.30 GMT


In [114]:
type_tag = soup.find(attrs = {"data-print-layout": "hide"}, class_ = "dcr-gfl9fa")
first_div = type_tag.find("div")
first_ul = first_div.find("ul")
tag_list = first_ul.find_all("li")
tag = [i.text for i in tag_list]
tag

['Liverpool', 'Manchester City', 'Jürgen Klopp', 'Pep Guardiola', 'news']

# Extract the whole data to a csv with features: title, description, content, time, tags

## Format the csv file to a csv file with a numerical order (don't need anymore)

In [135]:
# import csv

# input_file_name = 'guardian_links_2015_2023_order.csv'
# output_file_name = 'guardian_links_2015_2023_order.csv'

# with open(input_file_name, mode='r', newline='', encoding='utf-8') as input_file:
#     reader = csv.reader(input_file)
#     links_with_order = []

#     for i, row in enumerate(reader, start=1):
#         # Assuming each row has a single column which is the link
#         link = row[0]
#         links_with_order.append([i, link])  # Append order number and link

# with open(output_file_name, mode='w', newline='', encoding='utf-8') as output_file:
#     writer = csv.writer(output_file)
#     writer.writerow(['Order', 'Link'])  # Write header

#     for item in links_with_order:
#         writer.writerow(item)  # Write each row with order and link

# print(f"Formatted links saved to {output_file_name}")

Formatted links saved to guardian_links_2015_2023_order.csv


## Extract whole data

In [144]:
import csv
import requests
from bs4 import BeautifulSoup
import html
import time

# Function to extract content from a given URL
def extract_content(url):
    try:
        response = requests.get(url, timeout=10)  # Set a timeout for the request
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract title - primary method
        title_container = soup.find(attrs={"data-gu-name": "headline"})
        title_text = title_container.find('h1').get_text() if title_container and title_container.find('h1') else ''

        # Extract title - alternative method if primary method fails
        if not title_text:
            og_title = soup.find('meta', attrs={"property": "og:title"})
            title_text = og_title['content'] if og_title and 'content' in og_title.attrs else ''
        print(title_text)

        # Extract description
        meta_description = soup.find('meta', attrs={"name": "description"})
        description_text = ''
        if meta_description and 'content' in meta_description.attrs:
            description_content = meta_description['content']
            if '<' in description_content and '>' in description_content:
                decoded_description = html.unescape(description_content)
                description_soup = BeautifulSoup(decoded_description, 'html.parser')
                description_text = description_soup.get_text()
            else:
                description_text = description_content

        # Extract content
        content = soup.find(attrs={"data-gu-name": "body"})
        main_content = ''
        if content:
            newsletter_promotion = content.find('figure', class_='dcr-173mewl')
            if newsletter_promotion:
                newsletter_promotion.decompose()
            main_content = content.get_text().split("Explore more on these topics")[0].strip()

        # Extract time
        meta = soup.find(attrs={"data-gu-name": "meta"})
        date = meta.find("span", class_="dcr-u0h1qy").text if meta and meta.find("span", class_="dcr-u0h1qy") else ''
        if not date:
            date = soup.find("div", class_="dcr-18bzkx3").text if soup.find("div", class_="dcr-18bzkx3") else ''

        # Extract tags
        tags = []
        type_tag = soup.find(attrs={"data-print-layout": "hide"}, class_="dcr-gfl9fa")
        if type_tag:
            first_div = type_tag.find("div")
            first_ul = first_div.find("ul") if first_div else None
            tag_list = first_ul.find_all("li") if first_ul else []
            tags = [i.text for i in tag_list]

        return url, title_text, description_text, main_content, date, tags
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return url, '', '', '', '', []

input_file_name = 'guardian_links_2015_2023.csv'
output_file_name = 'extract_content_the_guardian.csv'
start_row = 1
end_row = 10

start_time = time.time()  # Start time measurement

with open(input_file_name, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    extracted_data = []

    for i, row in enumerate(reader, start=1):
        if i < start_row:
            continue  # Skip rows before the start_row
        if i > end_row:
            break  # Stop after processing up to the end_row
        link = row[0]
        extracted_data.append(extract_content(link))
        time.sleep(1)  # Delay between requests

with open(output_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL', 'Title', 'Description', 'Content', 'Time', 'Tags'])
    for data in extracted_data:
        writer.writerow(data)

end_time = time.time()  # End time measurement
print('Extraction complete. Data saved to', output_file_name)
print(f"Running time: {end_time - start_time:.2f} seconds")


One dead, six others shot at Calgary new year party
US warns Bahrain opposition leader's arrest could 'inflame tensions'
Widely different estimates of those still missing after Adriatic Sea ferry fire
New Year’s Eve stampede leaves shock and confusion in Shanghai
Iran and US tentatively agree on formula to reduce nuclear programme
Three held in India for alleged kidnap and gang-rape of Japanese woman
Smugglers abandon migrant ship off Italy in new tactic to force rescue
Body of missing Virginia yoga teacher found in Mexico ravine
Timeline: nurse Pauline Cafferkey’s return to UK and Ebola diagnosis
Families of climbers missing on New Zealand's Mount Cook told bodies may never be recovered
Extraction complete. Data saved to extract_content_the_guardian.csv
Running time: 12.48 seconds


### Merge many csv file to 1 csv file only with 188 thousand news articles 

In [4]:
import pandas as pd
import os

# Directory where your files are located
directory = "Content_the_Guardian_2015_2023"

# List of file names
file_names = [f"extract_content_the_guardian{i}.csv" for i in range(1, 9)]

# Initialize a list to store DataFrames
dfs = []

# Read each file and add to the list
for file in file_names:
    file_path = os.path.join(directory, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        dfs.append(df)
    else:
        print(f"File not found: {file}")

# Use pandas.concat to combine all DataFrames
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined data to a new CSV file
combined_df.to_csv(os.path.join(directory, "the_guardian_full.csv"), index=False)
